In [29]:
import cv2
import numpy as np
from scipy.signal import medfilt
import matplotlib.pyplot as plt

In [15]:
def motion_change_score(frame1, frame2):

   gray1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
   gray2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)

   flow = cv2.calcOpticalFlowFarneback(
       gray1, gray2, None,
       pyr_scale=0.5,
       levels=3,
       winsize=15,
       iterations=3,
       poly_n=5,
       poly_sigma=1.2,
       flags=0
   )
   mag, ang = cv2.cartToPolar(flow[...,0], flow[...,1])

   return np.array([np.mean(mag), np.std(mag), np.mean(ang), np.std(mag)])

In [23]:
def detect_scene(video_path, threshold=2.0, smooth_kernel=5, windows_size=50):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise ValueError("Не удалось открыть видео")

    frames = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)
    cap.release()

    if len(frames) < 2:
        fps = cv2.VideoCapture(video_path).get(cv2.CAP_PROP_FPS)
        return [], fps

    fps = cv2.VideoCapture(video_path).get(cv2.CAP_PROP_FPS)

    # Собираем признаки движения для каждой пары (i, i+1)
    motion_features = []
    for i in range(len(frames) - 1):
        feat = motion_change_score(frames[i], frames[i + 1])
        motion_features.append(feat)

    motion_features = np.array(motion_features)  # shape: (N-1, 4)

    # Считаем изменение движения между соседними переходами
    diffs = []
    for i in range(1, len(motion_features)):  # ← ИСПРАВЛЕНО: i, а не feat
        d = np.linalg.norm(motion_features[i] - motion_features[i - 1])
        diffs.append(d)

    if len(diffs) == 0:
        return [], fps

    diffs = np.array(diffs)

    # Сглаживание
    if smooth_kernel > 1 and len(diffs) >= smooth_kernel:
        diffs_smooth = medfilt(diffs, kernel_size=smooth_kernel)
    else:
        diffs_smooth = diffs

    thresholds = []
    for i in range(len(diffs_smooth)):
      start = max(0, i - windows_size // 2)
      end = min(len(diffs_smooth), windows_size + i // 2)
      local_mean = np.mean(diffs_smooth[start:end])
      local_std = np.std(diffs_smooth[start:end])
      thresholds.append(local_mean + threshold * local_std)

    thresholds = np.array(thresholds)


    print(f"[DEBUG] Max_thr: {max(thresholds):.3f}, min: {min(thresholds):.3f}")

    # Находим смены
    scene_change = []
    for i, d in enumerate(diffs_smooth):
        if d > thresholds[i]:
            scene_frame = i + 2  # i+2 — потому что diffs[i] соответствует переходу между (i+1) и (i+2)
            scene_change.append(scene_frame)

    return scene_change, fps

In [24]:
scene_change, fps = detect_scene('/content/drive/MyDrive/practical_data/man_in_costume.mp4', threshold=2.0)
print(f'Сцена меняется на кадрах:', scene_change)

[DEBUG] Max_thr: 25.485, min: 0.853
Сцена меняется на кадрах: [14, 15, 16, 17, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150]


/usr/local/lib/python3.12/dist-packages/numpy/_core/fromnumeric.py:3596: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.12/dist-packages/numpy/_core/_methods.py:138: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.12/dist-packages/numpy/_core/_methods.py:218: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/usr/local/lib/python3.12/dist-packages/numpy/_core/_methods.py:175: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/usr/local/lib/python3.12/dist-packages/numpy/_core/_methods.py:210: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)


In [25]:
for frame in scene_change:
  print(f'Кадр {frame} был на {(frame / fps):.2f} секунде')

Кадр 14 был на 0.47 секунде
Кадр 15 был на 0.50 секунде
Кадр 16 был на 0.53 секунде
Кадр 17 был на 0.57 секунде
Кадр 128 был на 4.27 секунде
Кадр 129 был на 4.30 секунде
Кадр 130 был на 4.33 секунде
Кадр 131 был на 4.37 секунде
Кадр 132 был на 4.40 секунде
Кадр 133 был на 4.43 секунде
Кадр 134 был на 4.47 секунде
Кадр 135 был на 4.50 секунде
Кадр 136 был на 4.53 секунде
Кадр 137 был на 4.57 секунде
Кадр 139 был на 4.63 секунде
Кадр 140 был на 4.67 секунде
Кадр 141 был на 4.70 секунде
Кадр 142 был на 4.73 секунде
Кадр 143 был на 4.77 секунде
Кадр 144 был на 4.80 секунде
Кадр 145 был на 4.83 секунде
Кадр 146 был на 4.87 секунде
Кадр 147 был на 4.90 секунде
Кадр 148 был на 4.93 секунде
Кадр 149 был на 4.97 секунде
Кадр 150 был на 5.00 секунде


In [30]:

def show_frame_at(video_path, frame_num):
    cap = cv2.VideoCapture(video_path)
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_num)
    ret, frame = cap.read()
    if ret:
        # Конвертируем BGR → RGB (потому что OpenCV использует BGR, а matplotlib — RGB)
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        plt.figure(figsize=(8, 6))
        plt.imshow(frame_rgb)
        plt.title(f"Кадр {frame_num}")
        plt.axis('off')
        plt.show()
    else:
        print(f"Кадр {frame_num} не найден")
    cap.release()

# Пример использования
for frame in scene_change:
 show_frame_at('/content/drive/MyDrive/practical_data/man_in_costume.mp4', frame)

Output hidden; open in https://colab.research.google.com to view.

In [12]:
scene_change, fps = detect_scene('/content/drive/MyDrive/practical_data/small_people.mp4', threshold=2.0)
print(f'Сцена меняется на кадрах:', scene_change)

Сцена меняется на кадрах: []


In [11]:
for frame in scene_change:
  print(f'Кадр {frame} был на {(frame / fps):.2f} секунде')

Кадр 5 был на 0.17 секунде
Кадр 6 был на 0.20 секунде
Кадр 7 был на 0.23 секунде
Кадр 8 был на 0.27 секунде
Кадр 14 был на 0.47 секунде
Кадр 15 был на 0.50 секунде
Кадр 16 был на 0.53 секунде
Кадр 17 был на 0.57 секунде
Кадр 18 был на 0.60 секунде
Кадр 19 был на 0.63 секунде
Кадр 20 был на 0.67 секунде
Кадр 21 был на 0.70 секунде
Кадр 22 был на 0.73 секунде
Кадр 23 был на 0.77 секунде
Кадр 24 был на 0.80 секунде
Кадр 25 был на 0.83 секунде
Кадр 26 был на 0.87 секунде
Кадр 29 был на 0.97 секунде
Кадр 57 был на 1.90 секунде
Кадр 58 был на 1.93 секунде
Кадр 59 был на 1.97 секунде
Кадр 342 был на 11.40 секунде
Кадр 343 был на 11.43 секунде
Кадр 345 был на 11.50 секунде
Кадр 346 был на 11.53 секунде
Кадр 376 был на 12.53 секунде
Кадр 377 был на 12.57 секунде
